In [1]:
from generator import *
from shredder import *
from fitter import *
from exec_tools import run
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from joblib import Parallel, delayed
from math import sqrt
import time
import itertools
#np.random.seed(10)



/Users/eilifm/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data_gen_dict = config_maker(2, 10, [.5], [[1,2]],[5])
data_gen_dict = config_maker(4, 10, [1,1,1], [],[])

In [4]:
estimates = list()
in_ci = list()
for iters in range(1000):
    fit_data, test_data, true_coeffs = generate_ind_model(
                                            data_gen_dict["dist_list"],
                                            data_gen_dict["main_coeffs"],
                                            data_gen_dict["interaction_coeffs"],
                                            data_gen_dict["interactions"],
                                            intercept=10,
                                            n=100,
                                            beta_sigma=.1
                                        )

    tmp_data = fit_data.copy(deep=True)
#     tmp_data = uniform_shred_cols(['x1'], .01, tmp_data)
#     tmp_data = tmp_data.dropna()
    model, metrics = fit_lm(tmp_data, test_data)
    b_estimate_results = beta_target_check(metrics, true_coeffs, as_dataframe=False)
    in_ci.append([b_estimate_results[key]['in_ci'] for key in b_estimate_results.keys()])
    
    estimates.append(model.params.values)

estimates_df = pd.DataFrame(estimates, columns=model.params.index)
in_ci_df = pd.DataFrame(in_ci)

In [5]:
print(estimates_df.mean())
print(pd.DataFrame(in_ci_df.mean().values, columns=['% on target'], index=estimates_df.mean().index))

const     9.970968
x1       10.014610
x2       10.023382
x3        9.994240
x4       10.020278
dtype: float64
       % on target
const        0.955
x1           0.948
x2           0.951
x3           0.955
x4           0.953


In [6]:
tmp_data = fit_data.copy(deep=True)
tmp_data = uniform_shred_cols(['x1'], 0, tmp_data)

test = pd.DataFrame(np.array([tmp_data['x1'], fit_data['x1']]).T, columns=['x1_altered', 'x1_original'])
print(test)
test[['x1_altered', 'x1_original']].assign(NE=test.x1_altered == test.x1_original)

    x1_altered  x1_original
0     0.193274     0.193274
1     0.459724     0.459724
2     0.425634     0.425634
3     0.936119     0.936119
4     0.805206     0.805206
5     0.647349     0.647349
6     0.498935     0.498935
7     0.500570     0.500570
8     0.213711     0.213711
9     0.515922     0.515922
10    0.500539     0.500539
11    0.301481     0.301481
12    0.180076     0.180076
13    0.530817     0.530817
14    0.902167     0.902167
15    0.402652     0.402652
16    0.382827     0.382827
17    0.607477     0.607477
18    0.291964     0.291964
19    0.016934     0.016934
20    0.044099     0.044099
21    0.156533     0.156533
22    0.683730     0.683730
23    0.057163     0.057163
24    0.066982     0.066982
25    0.798841     0.798841
26    0.900977     0.900977
27    0.120577     0.120577
28    0.717401     0.717401
29    0.875915     0.875915
..         ...          ...
70    0.435235     0.435235
71    0.237500     0.237500
72    0.098495     0.098495
73    0.840083     0

,x1_altered,x1_original,NE
0,0.193274,0.193274,True
1,0.459724,0.459724,True
2,0.425634,0.425634,True
3,0.936119,0.936119,True
4,0.805206,0.805206,True
5,0.647349,0.647349,True
6,0.498935,0.498935,True
7,0.500570,0.500570,True
8,0.213711,0.213711,True
9,0.515922,0.515922,True


In [7]:
print(true_coeffs)
print(model.summary())


{'x1': 10, 'x2': 10, 'x3': 10, 'x4': 10, 'const': 10}
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.965
Method:                 Least Squares   F-statistic:                     691.2
Date:                Mon, 26 Mar 2018   Prob (F-statistic):           2.73e-69
Time:                        08:49:48   Log-Likelihood:                -134.93
No. Observations:                 100   AIC:                             279.9
Df Residuals:                      95   BIC:                             292.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [8]:
model.params.values


array([10.1716721 ,  9.86903996,  9.88160368, 10.13238304,  9.65141119])

# Rewrite of the Invert 
It seems that it still only works to invert if only one column is missing. If more than one is missing then you're still out of luck. 

In [9]:
#data_gen_dict = config_maker(3, 5, [.4, .4], [[1,2], [1,3], [2,3]],[5, 10, 8])
data_gen_dict = config_maker(
    2,
    5,
    [.4],
    [
        [1,2]
    ], 
    [4])
#data_gen_dict = config_maker(1, 10, [], [],[])
fit_data, test_data, true_coeffs = generate_ind_model(
                                        data_gen_dict["dist_list"],
                                        data_gen_dict["main_coeffs"],
                                        data_gen_dict["interaction_coeffs"],
                                        data_gen_dict["interactions"],
                                        intercept=1,
                                        n=100,
                                        beta_sigma=.2
                                    )

# Drop some data
#tmp_data = uniform_shred_cols(['x2'], .5, fit_data)
tmp_data = fit_data.copy()
tmp_data = uniform_shred_cols(['x2'], .4, tmp_data)
tmp_data = uniform_shred_cols(['x1'], .1, tmp_data)


import importlib
import fixer
importlib.reload(fixer)

inverted,  fitted = fixer.olsinv_singlex(tmp_data, 'x2', test_mode=False)

print(tmp_data.head(10))
print(inverted.head(10))

tmp_data.loc[inverted.index, 'x2'] = inverted

inverted,  fitted = fixer.olsinv_singlex(tmp_data, 'x1', test_mode=False)

print(tmp_data.head(10))
print(inverted.head(10))

tmp_data.loc[inverted.index, 'x1'] = inverted
print(tmp_data.head(40))

         x1        x2     x1:x2         y
0  0.286904  0.128590  0.036893  3.311838
1  0.100340  0.607865  0.060993  1.671210
2  0.183216       NaN  0.179043  5.448216
3  0.302661  0.864875  0.261764  4.766399
4  0.653058  0.255086  0.166586  6.179515
5  0.698821       NaN  0.319239  6.083408
6  0.144180  0.519456  0.074895  0.493273
7  0.456659       NaN  0.224673  5.870519
8  0.269666       NaN  0.185149  5.127545
9  0.873200  0.602734  0.526308  9.658790
2     1.340748
5     0.320938
7     0.759343
8     0.979902
13    1.104049
15    0.735651
17    0.243798
19    0.272661
20    0.132657
22    0.591425
Name: y, dtype: float64
         x1        x2     x1:x2         y
0  0.286904  0.128590  0.036893  3.311838
1  0.100340  0.607865  0.060993  1.671210
2  0.183216  1.340748  0.179043  5.448216
3  0.302661  0.864875  0.261764  4.766399
4  0.653058  0.255086  0.166586  6.179515
5  0.698821  0.320938  0.319239  6.083408
6  0.144180  0.519456  0.074895  0.493273
7  0.456659  0.759343  0.224